In [ ]:
import pandas as pd

In [ ]:
excel_file_path = '1.xlsx'  # loading
csv_file_path = '1.csv'  # desired

In [ ]:
df = pd.read_excel(excel_file_path)

In [ ]:
df.to_csv(csv_file_path, index=False)

In [ ]:
df.head(2)

In [ ]:
df.columns

In [ ]:
selected_columns = ['Class', 'Severity', 'Alarm State', 'Alert State', 'Alert Target', 'Target Pod', 'Last Updated (US/Eastern)', 'Created (US/Eastern)']
df_selected = df[selected_columns]
df_selected.head(2)

In [ ]:
df_selected.to_csv('filtered.csv', index=False)
df_selected.columns

In [ ]:
df_selected.dtypes

In [ ]:
df_selected.isnull().sum()

In [ ]:
rows_any_null = df_selected[df_selected.isnull().any(axis=1)]
rows_any_null

In [ ]:
fill_value = 'No Data'
df_selected['Alert Target'] = df_selected['Alert Target'].fillna(fill_value)

In [ ]:
rows_any_null = df_selected[df_selected.isnull().any(axis=1)]
rows_any_null

In [ ]:
df_selected.isnull().sum()

In [ ]:
df_selected.head(2)

In [ ]:
new_column_names = ['class', 'severity', 'alarm_state', 'alert_state', 'alert_target', 'target_pod', 'last_updated', 'created']
df_selected.columns = new_column_names

In [ ]:
df_selected.columns

In [ ]:
from pytz import timezone

In [ ]:
configurable_tz = "US/Eastern"

In [ ]:
# Function to convert string to datetime with timezone
def convert_to_datetime_with_tz(date_str, tz):
    try:
        # Convert string to datetime
        datetime_obj = pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S:%f %Z', errors='coerce')
        # Localize to the given timezone
        return datetime_obj.tz_localize(None).tz_localize(tz)
    except Exception as e:
        print(f"Error converting {date_str}: {e}")
        return None

In [ ]:
date_column = 'created'
df_selected[date_column] = df_selected[date_column].apply(lambda x: convert_to_datetime_with_tz(x, timezone(configurable_tz)))

In [ ]:
df_selected.dtypes

In [ ]:
date_column = 'last_updated'
df_selected[date_column] = df_selected[date_column].apply(lambda x: convert_to_datetime_with_tz(x, timezone(configurable_tz)))

In [ ]:
df_selected.dtypes

In [ ]:
df_selected.head(2)

In [ ]:
date_column = 'created'
df_selected[date_column] = pd.to_datetime(df_selected[date_column], format='%Y-%m-%d %H:%M:%S:%f %Z', errors='coerce')

In [ ]:
df_selected.head(2)

In [ ]:
# Round the datetime to the nearest hour
df_selected['RoundedHour'] = df_selected[date_column].dt.floor('H')

In [ ]:
df_selected.head(2)

In [ ]:
# Group by the rounded hour and count occurrences
alert_counts = df_selected.groupby('RoundedHour').size().reset_index(name='AlertCount')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Prepare the plot
fig, ax = plt.subplots(figsize=(12, 6))

# Plot the data
ax.plot(alert_counts['RoundedHour'], alert_counts['AlertCount'], marker='o', linestyle='-', label='Alert Count')

# Create the checkered grid layout
start_time = alert_counts['RoundedHour'].min()
end_time = alert_counts['RoundedHour'].max()

# Define the grid intervals
hourly_intervals = pd.date_range(start=start_time, end=end_time, freq='H')  # Hourly ticks
daily_intervals = pd.date_range(start=start_time, end=end_time, freq='D')  # Daily ticks

# Add hour numbers (0-23) for each day
hour_labels = [hour % 24 for hour in range(len(hourly_intervals))]  # Cycle through 0-23

# Set ticks for a fine grid
ax.set_xticks(hourly_intervals, minor=True)  # Minor ticks for each hour
ax.set_xticks(daily_intervals, minor=False)  # Major ticks for each day
ax.set_xticklabels([label.strftime('%Y-%m-%d') for label in daily_intervals], rotation=45)  # Show dates on major ticks
ax.set_xticks(hourly_intervals, minor=True)  # Set minor ticks for hours
ax.set_xticklabels(hour_labels, minor=True)  # Add hour numbers for minor ticks

# Adjust the font size of the hour numbers (minor ticks)
hour_font_size = 4  # Change this to adjust the font size
ax.tick_params(axis='x', which='minor', labelsize=hour_font_size)

# Enable gridlines
ax.grid(which='minor', linestyle=':', linewidth=0.5, color='gray')  # Hourly grid
ax.grid(which='major', linestyle='--', linewidth=0.8, color='black')  # Daily grid

# Add labels and title
ax.set_title('Alert Count per Hour with Adjustable Hour Number Size')
ax.set_xlabel('Time')
ax.set_ylabel('Alert Count')
plt.legend()

# Tight layout and show
plt.tight_layout()
plt.show()

In [ ]:
df_selected.head(2)

In [ ]:
# Group by 'RoundedHour' and 'Target Pod' and count occurrences
target_pod_counts = df_selected.groupby(['RoundedHour', 'target_pod']).size().reset_index(name='AlertCount')

# Pivot the data for plotting (Target Pods as separate lines)
pivoted_data = target_pod_counts.pivot(index='RoundedHour', columns='target_pod', values='AlertCount').fillna(0)

In [ ]:
# Plot the data
fig, ax = plt.subplots(figsize=(36, 18))

for target_pod in pivoted_data.columns:
    ax.plot(
        pivoted_data.index, 
        pivoted_data[target_pod], 
        marker='o', 
        linestyle='-', 
        label=f'Target Pod: {target_pod}'
    )

# Create the checkered grid layout
start_time = pivoted_data.index.min()
end_time = pivoted_data.index.max()

# Define the grid intervals
hourly_intervals = pd.date_range(start=start_time, end=end_time, freq='H')  # Hourly ticks
daily_intervals = pd.date_range(start=start_time, end=end_time, freq='D')  # Daily ticks

# Add hour numbers (0-23) for each day
hour_labels = [hour % 24 for hour in range(len(hourly_intervals))]  # Cycle through 0-23

# Set ticks for a fine grid
ax.set_xticks(hourly_intervals, minor=True)  # Minor ticks for each hour
ax.set_xticks(daily_intervals, minor=False)  # Major ticks for each day
ax.set_xticklabels([label.strftime('%Y-%m-%d') for label in daily_intervals], rotation=45)  # Show dates on major ticks
ax.set_xticks(hourly_intervals, minor=True)  # Set minor ticks for hours
ax.set_xticklabels(hour_labels, minor=True)  # Add hour numbers for minor ticks

# Adjust the font size of the hour numbers (minor ticks)
hour_font_size = 10  # Change this to adjust the font size
ax.tick_params(axis='x', which='minor', labelsize=hour_font_size)

# Enable gridlines
ax.grid(which='minor', linestyle=':', linewidth=0.5, color='gray')  # Hourly grid
ax.grid(which='major', linestyle='--', linewidth=0.8, color='black')  # Daily grid

# Add labels and title
ax.set_title('Alert Count per Hour by Target Pod')
ax.set_xlabel('Time')
ax.set_ylabel('Alert Count')
plt.legend(title='Target Pods')

# Tight layout and show
plt.tight_layout()
plt.show()

In [ ]:
df_selected.head(2)

In [ ]:
# Group by 'RoundedHour' and 'Alert Target' and count occurrences
alert_target_counts = df_selected.groupby(['RoundedHour', 'alert_target']).size().reset_index(name='AlertCount')

# Pivot the data for plotting (Alert Targets as separate lines)
pivoted_data = alert_target_counts.pivot(index='RoundedHour', columns='alert_target', values='AlertCount').fillna(0)


In [ ]:
import numpy as np

In [ ]:
# Plot the bar graph
fig, ax = plt.subplots(figsize=(42, 21))

# Define bar positions
bar_width = 0.1  # Width of each bar
x_indices = np.arange(len(pivoted_data.index))  # X positions for the bars
offset = 0  # Offset for each Alert Target

# Add bars for each Alert Target
for idx, alert_target in enumerate(pivoted_data.columns):
    ax.bar(
        x_indices + offset, 
        pivoted_data[alert_target], 
        width=bar_width, 
        label=f'Alert Target: {alert_target}'
    )
    offset += bar_width  # Shift the next bar group horizontally

# Format the x-axis
ax.set_xticks(x_indices + bar_width * (len(pivoted_data.columns) - 1) / 2)  # Center ticks
ax.set_xticklabels(pivoted_data.index.strftime('%Y-%m-%d %H:%M'), rotation=45)

# Add gridlines and labels
ax.grid(axis='y', linestyle='--', linewidth=0.5, color='gray')
ax.set_title('Alert Count per Hour by Alert Target (Bar Graph)')
ax.set_xlabel('Time')
ax.set_ylabel('Alert Count')
ax.legend(title='Alert Targets')

# Tight layout and show
plt.tight_layout()
plt.show()